[H2O AutoML](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html) is an automated machine learning meta-algorithm that is part of the [H2O software library](http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/intro.html#what-is-h2o). (It shold not be confused with [H2O DriverlessAI](https://www.h2o.ai/products/h2o-driverless-ai/), which is a commercial product and built from an entirely different code base.) H2O’s AutoML can be used for automating the machine learning workflow, which includes automatic training and tuning of many models within a user-specified time-limit. Stacked Ensembles – one based on all previously trained models, another one on the best model of each family – will be automatically trained on collections of individual models to produce highly predictive ensemble models which, in most cases, will be the top performing models in the AutoML Leaderboard.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:
import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

In [ ]:
train = h2o.import_file("../input/melanoma-train-test-creator/train_meta_size_2.csv")
test = h2o.import_file("../input/melanoma-train-test-creator/test_meta_size_2.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
x = test.columns
y = 'target'

In [ ]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()

In [ ]:
aml = H2OAutoML(max_models=10000, seed=47, max_runtime_secs=14400)
aml.train(x=x, y=y, training_frame=train)

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

In [ ]:
# The leader model is stored here
aml.leader

In [ ]:
preds = aml.predict(test)

In [ ]:
preds['p1'].as_data_frame().values.flatten().shape

In [ ]:
preds

In [ ]:
sample_submission = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
sample_submission.head()

In [ ]:
sample_submission['target'] = preds['p1'].as_data_frame().values
sample_submission.to_csv('submission.csv', index=False)